In [4]:
import io
import json

from pathlib import Path
from urllib.parse import urlparse, parse_qs

import pandas as pd
import numpy as np

import requests
import clipboard

from PIL import Image
from jinja2 import Template

In [6]:
module_number = 3
module_title = 'Orchestration and ML Pipelines'
module_folder = Path('..') / '03-orchestration'

meta_json_file = module_folder / 'meta.json'
playlist_id = 'PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK'

In [12]:
module_number = 5
module_title = 'ML Monitoring'
module_folder = Path('..') / '05-monitoring'

meta_json_file = module_folder / 'meta.json'
playlist_id = 'PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK'

In [3]:
module_number = 6
module_title = 'Best Practices'
module_folder = Path('..') / '06-best-practices'

meta_json_file = module_folder / 'meta.json'
playlist_id = 'PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK'

In [4]:
module_number = 4
module_title = 'Model Deployment'
module_folder = Path('..') / '04-deployment'

meta_json_file = module_folder / 'meta.json'
playlist_id = 'PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK'

## Prepare `meta.json` info

In [13]:
text = """
module_number	module_title	unit_number	unit_title	full_title	youtube
5	Monitoring	1	Intro to ML monitoring	MLOps Zoomcamp 5.1 - Intro to ML monitoring	https://www.youtube.com/watch?v=SQ0jBwd_3kk
5	Monitoring	2	Environment setup	MLOps Zoomcamp 5.2 - Environment setup	https://www.youtube.com/watch?v=yixA3C1xSxc
5	Monitoring	3	Prepare reference and model	MLOps Zoomcamp 5.3 - Prepare reference and model	https://www.youtube.com/watch?v=IjNrkqMYQeQ
5	Monitoring	4	Evidently metrics calculation	MLOps Zoomcamp 5.4 - Evidently metrics calculation	https://www.youtube.com/watch?v=kP3lzh_HfWY
5	Monitoring	5	Dummy monitoring	MLOps Zoomcamp 5.5 - Dummy monitoring	https://www.youtube.com/watch?v=s3G4PMsOMOA
5	Monitoring	6	Data quality monitoring	MLOps Zoomcamp 5.6 - Data quality monitoring	https://www.youtube.com/watch?v=fytrmPbcLhI
5	Monitoring	7	Save Grafana Dashboard	MLOps Zoomcamp 5.7 - Save Grafana Dashboard	https://www.youtube.com/watch?v=-c4iumyZMyw
5	Monitoring	8	Debugging with test suites and reports	MLOps Zoomcamp 5.8 - Debugging with test suites and reports	https://www.youtube.com/watch?v=sNSk3ojISh8""".strip()

In [14]:
df = pd.read_csv(io.StringIO(text), delimiter='\t')

In [15]:
df['youtube'] = df['youtube'].fillna('')

In [16]:
df_units = df[['unit_number', 'unit_title', 'youtube']]
units = df_units \
    .rename(columns={'unit_number': 'number', 'unit_title': 'title'}) \
    .to_dict(orient='records')
units

In [17]:
meta = {
    'module': {
        'number': module_number,
        'title': module_title
    },
    'units': units
}

In [18]:
meta

In [19]:
module_folder.mkdir(parents=True, exist_ok=True)

with open(meta_json_file, 'wt') as f_out:
    json.dump(meta, f_out, indent=2)

In [20]:
!head {meta_json_file}

## Generate page

module_folder = Path('..') / '05-monitoring'
meta_json_file = module_folder / 'meta.json'
playlist_id = 'PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK'

In [21]:
with meta_json_file.open('rt') as f_in:
    meta = json.load(f_in)

In [22]:
module_info = meta['module']
units = meta['units']

In [23]:
module_info

In [24]:
images_folder = module_folder / 'images'
images_folder.mkdir(parents=True, exist_ok=True)

In [25]:
template_string = """
## {{ module_number }}.{{ unit_number }} {{ unit_title }}

{% if youtube %}<a href="{{ youtube }}">
  <img src="{{ thumbnail }}">
</a>{% endif %}{% if not youtube %}COMING SOON{% endif %}


""".lstrip()

template = Template(template_string)

In [28]:
def download_thumbnail(video, module, unit, folder):
    if type(unit) in [int, np.int64]:
        thumbnail_file = f'thumbnail-{module}-{unit:02d}.jpg'
    else:
        thumbnail_file = f'thumbnail-{module}-{unit}.jpg'

    thumbnail_file = folder / thumbnail_file

    if thumbnail_file.exists():
        print(f'{thumbnail_file} exists')
        return thumbnail_file

    video_id = parse_qs(urlparse(video).query)['v'][0]
    print(f'processing video {video_id}...')
    thumbnail_url = f'https://img.youtube.com/vi/{video_id}/0.jpg'

    response = requests.get(thumbnail_url)
    thumbnail = Image.open(io.BytesIO(response.content))
    w_img, h_img = thumbnail.size

    play = Image.open(Path('../images/play.png'))
    w_play, h_play = play.size
    
    x0 = w_img // 2 - w_play // 2
    y0 = h_img // 2 - h_play // 2

    thumbnail.paste(play, (x0, y0), play)
    thumbnail.save(thumbnail_file, quality=90)

    print('saved to', thumbnail_file)

    return thumbnail_file

In [29]:
module_number = module_info['number']

parts = []

for unit in units:
    unit_number = unit['number']

    params = {
        'module_number': module_info['number'],
        'module_name': module_info['title'],
        'unit_number': unit['number'],
        'unit_title': unit['title']          
    }

    if 'youtube' in unit:
        youtube = unit['youtube']
        if len(youtube) and youtube.startswith('https'):
            thumbnail = download_thumbnail(youtube, module_number, unit_number, images_folder)
            thumbnail_path = '/'.join(thumbnail.parts[2:])
            params['youtube'] = f'{youtube}&list={playlist_id}'
            params['thumbnail'] = thumbnail_path
        
    template_string = template.render(params)
    print(template_string)
    parts.append(template_string)

In [30]:
prefix = f"""
# {module_info['number']}. {module_info['title']} 
""".strip()


final_result = '\n\n'.join([prefix] + parts)

In [31]:
clipboard.copy(final_result)